In [1]:
import torch
from torch.utils.data import DataLoader
from scripts.pums_downloader import datasets

from scripts.pums_sgd import PumsModule, load_pums, problem, evaluate, printf

In [2]:
train_loader = DataLoader(load_pums(datasets[1]), batch_size=1000, shuffle=True)
test_loader = DataLoader(load_pums(datasets[1]), batch_size=1000)

In [3]:
model = PumsModule(len(problem['predictors']), 2)
optimizer = torch.optim.SGD(model.parameters(), .1)

gradients = dict((name, []) for name, _ in model.named_parameters())
# printf(f"acc\t|\tloss")

for batch in train_loader:
    loss = model.loss(batch)
    loss.backward()
    # gradients.append(list(model.linear1.parameters()))
    for name, param in model.named_parameters():
        if param.requires_grad:
            grad = param.grad.detach().clone()
            gradients[name].append(grad)
    optimizer.step()
    # print(model.linear1.weight.grad)
    # print(model.linear1.bias.grad)

    optimizer.zero_grad()

    accuracy, loss = evaluate(model, test_loader)
    # printf(f"{accuracy.item():.2f}\t|\t{loss.item():.2f}", force=True)

tensor([[ 2.6140,  3.6927,  0.0610],
        [-3.2760, -4.5659, -0.0757],
        [ 2.6747,  3.7278,  0.0618],
        [-0.5048, -1.3536, -0.0143],
        [-0.6172, -1.0445, -0.0159]])
tensor([ 0.0383, -0.0477,  0.0390, -0.0093, -0.0098])
tensor([[-1.6793, -2.5172, -3.2861, -0.0552, -0.3641],
        [ 1.6793,  2.5172,  3.2861,  0.0552,  0.3641]])
tensor([-0.0696,  0.0696])
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00],
        [4.4678e+01, 1.1198e+02, 2.2036e+00],
        [1.3740e+00, 6.4031e+00, 1.0173e-01],
        [2.0535e+01, 5.2545e+01, 1.0229e+00],
        [9.6655e+00, 2.4318e+01, 4.7765e-01]])
tensor([0.0000, 1.1119, 0.0515, 0.5156, 0.2410])
tensor([[  0.0000,  94.7019,   7.1778,  16.1010,  20.2485],
        [  0.0000, -94.7019,  -7.1778, -16.1010, -20.2485]])
tensor([ 0.9350, -0.9350])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0., 0., 0., 0.])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0.,

In [ ]:
import numpy as np
import opendp.smartnoise.core as sn

with sn.Analysis():
    grads = [(x.mean().item(),) for x in gradients['linear1.weight']]
    dataset = sn.literal(value=grads, value_public=False)
    typed = sn.cast(dataset, atomic_type={
        bool: 'bool', float: 'float', int: 'int', str: 'str'
    }[float], true_label=True, lower=0, upper=10)
    resized = sn.resize(typed, number_columns=2, lower=0., upper=10.)
    data = sn.to_dataframe(resized, names=['linear1.weight'])
    print(data)
    candidates = [0., 3., 4., 7., 10., 12.]

    median_scores = sn.median(
        data,
        candidates=candidates,
        data_rows=len(gradients))
    print(median_scores.value)
    # dp_median = sn.exponential_mechanism(median_scores, candidates=candidates, privacy_usage={"epsilon": 1.})
    # print(dp_median.value)

In [ ]:
for batch in train_loader:
    for x in batch:
        print(x.size())
        break

In [96]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.grad.mean())

linear1.weight tensor(-1.0224, grad_fn=<MeanBackward0>)
linear1.bias tensor(-0.0673, grad_fn=<MeanBackward0>)
linear2.weight tensor(0.0010, grad_fn=<MeanBackward0>)
linear2.bias tensor(0.0321, grad_fn=<MeanBackward0>)


In [98]:
for param in model.parameters():
    print(param.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0., 0., 0., 0.])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([0., 0.])


In [48]:
for x,y in model.state_dict().items():
    print(x)
    print(y)

linear1.weight
tensor([[-0.8030, -1.3712,  0.4771],
        [-0.9598, -1.3771,  0.3245],
        [ 0.2147, -0.4524, -0.4112],
        [-0.5053, -0.2353,  0.4764],
        [-0.2368, -0.5467, -0.3327]])
linear1.bias
tensor([-0.2324,  0.0582, -0.3123, -0.5034,  0.2179])
linear2.weight
tensor([[-1.7393, -1.7952, -0.1300,  0.2737,  0.1703],
        [ 1.5121,  2.1671,  0.0881, -0.0228, -0.1676]])
linear2.bias
tensor([-0.7910,  0.6299])


In [55]:
loss.grad

In [76]:
model.linear1.weight

Parameter containing:
tensor([[ 0.5281,  0.1180, -0.0080],
        [ 0.3511,  0.2915, -0.0006],
        [ 0.0574,  0.2466,  0.2036],
        [-0.1973, -0.3723,  0.3213],
        [ 0.5404,  0.2739, -0.4443]], requires_grad=True)